In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ml_projects/drone/videos1.zip

Archive:  /content/drive/MyDrive/ml_projects/drone/videos1.zip
   creating: videos/
   creating: videos/flood/
  inflating: videos/flood/flood6.mp4  
  inflating: videos/flood/flood0.webm  
  inflating: videos/flood/flood8.webm  
  inflating: videos/flood/flood9.webm  
  inflating: videos/flood/fllood3.mp4  
  inflating: videos/flood/flood1.webm  
  inflating: videos/flood/flood2.webm  
  inflating: videos/flood/fllood4.mp4  
  inflating: videos/flood/flood5.mp4  
  inflating: videos/flood/flood7.webm  
   creating: videos/agri/
  inflating: videos/agri/agri0.webm  
  inflating: videos/agri/agri5.webm  
  inflating: videos/agri/agri1.webm  
  inflating: videos/agri/agri6.webm  
  inflating: videos/agri/agri4.webm  
  inflating: videos/agri/agri3.webm  
  inflating: videos/agri/agri7.webm  
  inflating: videos/agri/agri8.webm  
  inflating: videos/agri/agri9.webm  
  inflating: videos/agri/agri2.webm  
   creating: videos/drought/
  inflating: videos/drought/265279983.mp4  
  inflating:

## gabor filter + size zone matrix and lukas_kanade

### gabor filter + size zone matrix

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import os

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(y_true, y_pred):
    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    # Print the results
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1:.2f}")

In [ ]:
def showimage(myimage, figsize=[10,10]):
    if (myimage.ndim>2):
        myimage = myimage[:,:,::-1] 
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(myimage, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([]) 
    plt.show()

In [ ]:
def create_gaborfilter():
    filters = []
    num_filters = 16
    ksize = 35 
    sigma = 3.0 
    lambd = 10.0
    gamma = 0.5
    psi = 0  
    for theta in np.arange(0, np.pi, np.pi / num_filters): 
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_64F)
        kern /= 1.0 * kern.sum()
        filters.append(kern)
    return filters

In [ ]:
def apply_filter(img, filters):
    newimage = np.zeros_like(img)
    depth = -1
    for kern in filters:
        image_filter = cv2.filter2D(img, depth, kern)
        np.maximum(newimage, image_filter, newimage)
    return newimage

In [ ]:
def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):  #iterate through each file 
        #print(image)
        
        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.
        
        img = dataset[image, :]
        
        GLCM = greycomatrix(img, [1], [0])       
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df['Energy'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df['Corr'] = GLCM_corr       
        GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
        df['Diss_sim'] = GLCM_diss       
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df['Homogen'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df['Contrast'] = GLCM_contr


        GLCM2 = greycomatrix(img, [3], [0])       
        GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
        df['Energy2'] = GLCM_Energy2
        GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
        df['Corr2'] = GLCM_corr2       
        GLCM_diss2 = greycoprops(GLCM2, 'dissimilarity')[0]
        df['Diss_sim2'] = GLCM_diss2       
        GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
        df['Homogen2'] = GLCM_hom2       
        GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
        df['Contrast2'] = GLCM_contr2

        GLCM3 = greycomatrix(img, [5], [0])       
        GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
        df['Energy3'] = GLCM_Energy3
        GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
        df['Corr3'] = GLCM_corr3       
        GLCM_diss3 = greycoprops(GLCM3, 'dissimilarity')[0]
        df['Diss_sim3'] = GLCM_diss3       
        GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
        df['Homogen3'] = GLCM_hom3       
        GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
        df['Contrast3'] = GLCM_contr3

        GLCM4 = greycomatrix(img, [0], [np.pi/4])       
        GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
        df['Energy4'] = GLCM_Energy4
        GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
        df['Corr4'] = GLCM_corr4       
        GLCM_diss4 = greycoprops(GLCM4, 'dissimilarity')[0]
        df['Diss_sim4'] = GLCM_diss4       
        GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
        df['Homogen4'] = GLCM_hom4       
        GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
        df['Contrast4'] = GLCM_contr4
        
        GLCM5 = greycomatrix(img, [0], [np.pi/2])       
        GLCM_Energy5 = greycoprops(GLCM5, 'energy')[0]
        df['Energy5'] = GLCM_Energy5
        GLCM_corr5 = greycoprops(GLCM5, 'correlation')[0]
        df['Corr5'] = GLCM_corr5       
        GLCM_diss5 = greycoprops(GLCM5, 'dissimilarity')[0]
        df['Diss_sim5'] = GLCM_diss5       
        GLCM_hom5 = greycoprops(GLCM5, 'homogeneity')[0]
        df['Homogen5'] = GLCM_hom5       
        GLCM_contr5 = greycoprops(GLCM5, 'contrast')[0]
        df['Contrast5'] = GLCM_contr5
        
        #Add more filters as needed
        #entropy = shannon_entropy(img)
        #df['Entropy'] = entropy

        
        #Append features from current image to the dataset
        image_dataset = image_dataset.append(df)
        
    return image_dataset

### Lukas kanade

In [ ]:
dir='videos'
categories=['flood', 'agri', 'drought']
videos = []
labels = []
for category in categories:
    path=os.path.join(dir,category)
    label=categories.index(category)
    for img in os.listdir(path):
        imgpath=os.path.join(path,img)
        videos.append(imgpath)
        labels.append(label)

In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

data = []
dynamic_features = []
frame_labels = []

for i in range(len(videos)):
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )

    cap = cv2.VideoCapture(videos[i])

    ret, old_frame = cap.read()

    # filters

    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    old_frame=cv2.resize(old_gray,(50,50))
    gfilters = create_gaborfilter()
    old_frame = apply_filter(old_frame, gfilters)
    
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # filters

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame=cv2.resize(frame_gray,(50,50))
        gfilters = create_gaborfilter()
        frame = apply_filter(frame, gfilters)
        data.append(frame)


        p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame, frame, p0, None, **lk_params)

        flow_magnitude = np.sqrt(p1[:,:,0]**2 + p1[:,:,1]**2)
        dynamic_features.append(np.mean(flow_magnitude))
        frame_labels.append(labels[i])
        old_gray = frame_gray.copy()

    cap.release()

In [ ]:
data = np.array(data)

In [ ]:
data = feature_extractor(data)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
dynamic_features = pd.DataFrame(dynamic_features)

In [ ]:
dynamic_features, data

(                0
 0      244.864029
 1      244.864029
 2      244.864029
 3      244.864029
 4      244.864029
 ...           ...
 18814  366.095398
 18815  366.095398
 18816  366.095398
 18817  366.095398
 18818  366.095398
 
 [18819 rows x 1 columns],
       Energy      Corr  Diss_sim   Homogen   Contrast   Energy2     Corr2  \
 0   0.087320  0.996886  2.544490  0.456452  18.288980  0.082265  0.978565   
 0   0.087198  0.996846  2.565714  0.455372  18.468571  0.082276  0.978303   
 0   0.087499  0.996771  2.605714  0.451756  18.844898  0.082279  0.977764   
 0   0.087391  0.996766  2.596735  0.455397  18.875102  0.082235  0.977764   
 0   0.087602  0.996713  2.608571  0.458408  19.110612  0.082382  0.977537   
 ..       ...       ...       ...       ...        ...       ...       ...   
 0   0.083627  0.995313  2.014286  0.489966  11.023265  0.072968  0.969450   
 0   0.083593  0.995883  1.842449  0.512455   9.756735  0.072645  0.971897   
 0   0.082413  0.994918  2.021633  0.4970

In [ ]:
df = pd.concat([dynamic_features.reset_index(drop=True), data.reset_index(drop=True)], axis=1)

In [ ]:
df

,0,Energy,Corr,Diss_sim,Homogen,Contrast,Energy2,Corr2,Diss_sim2,Homogen2,...,Energy4,Corr4,Diss_sim4,Homogen4,Contrast4,Energy5,Corr5,Diss_sim5,Homogen5,Contrast5
0,244.864029,0.087320,0.996886,2.544490,0.456452,18.288980,0.082265,0.978565,6.833191,0.274090,...,0.114883,1.0,0.0,1.0,0.0,0.114883,1.0,0.0,1.0,0.0
1,244.864029,0.087198,0.996846,2.565714,0.455372,18.468571,0.082276,0.978303,6.885957,0.277345,...,0.114939,1.0,0.0,1.0,0.0,0.114939,1.0,0.0,1.0,0.0
2,244.864029,0.087499,0.996771,2.605714,0.451756,18.844898,0.082279,0.977764,7.000851,0.271788,...,0.114937,1.0,0.0,1.0,0.0,0.114937,1.0,0.0,1.0,0.0
3,244.864029,0.087391,0.996766,2.596735,0.455397,18.875102,0.082235,0.977764,6.971489,0.277667,...,0.115121,1.0,0.0,1.0,0.0,0.115121,1.0,0.0,1.0,0.0
4,244.864029,0.087602,0.996713,2.608571,0.458408,19.110612,0.082382,0.977537,6.986809,0.276864,...,0.115071,1.0,0.0,1.0,0.0,0.115071,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18814,366.095398,0.083627,0.995313,2.014286,0.489966,11.023265,0.072968,0.969450,5.091489,0.296604,...,0.132692,1.0,0.0,1.0,0.0,0.132692,1.0,0.0,1.0,0.0
18815,366.095398,0.083593,0.995883,1.842449,0.512455,9.756735,0.072645,0.971897,4.754468,0.318486,...,0.132002,1.0,0.0,1.0,0.0,0.132002,1.0,0.0,1.0,0.0
18816,366.095398,0.082413,0.994918,2.021633,0.497078,11.810204,0.072047,0.968172,5.101277,0.309396,...,0.130780,1.0,0.0,1.0,0.0,0.130780,1.0,0.0,1.0,0.0
18817,366.095398,0.080837,0.993854,2.190204,0.482406,14.062857,0.070920,0.962646,5.566809,0.291466,...,0.128288,1.0,0.0,1.0,0.0,0.128288,1.0,0.0,1.0,0.0


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(df, frame_labels, test_size=0.25,random_state=42)

In [ ]:
def evaluate

In [ ]:
clf_svc = SVC()
clf_svc.fit(X_train, Y_train)
y_pred_svc = clf_svc.predict(X_test)
accuracy_svc = accuracy_score(Y_test, y_pred_svc)


clf_gaussian = GaussianNB()
clf_gaussian.fit(X_train, Y_train)
y_pred_gaussian = clf_gaussian.predict(X_test)
accuracy_gaussian = accuracy_score(Y_test, y_pred_gaussian)

clf_logistic = LogisticRegression()
clf_logistic.fit(X_train, Y_train)
y_pred_logistic = clf_logistic.predict(X_test)

print("===SVC===")
evaluate_model(Y_test, y_pred_svc)
print("===Gaussian===")
evaluate_model(Y_test, y_pred_gaussian)
print("===Logistic===")
evaluate_model(Y_test, y_pred_logistic)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

===SVC===
Accuracy: 0.70
Precision: 0.70
Recall: 0.70
F1 score: 0.70
===Gaussian===
Accuracy: 0.60
Precision: 0.60
Recall: 0.60
F1 score: 0.60
===Logistic===
Accuracy: 0.69
Precision: 0.69
Recall: 0.69
F1 score: 0.69


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


## local binary + Size Zone and lukas kanade

### local binary + Size Zone

In [ ]:
def get_pixel(img, center, x, y):
     
    new_value = 0
     
    try:
        if img[x][y] >= center:
            new_value = 1
             
    except:
        pass
     
    return new_value

In [ ]:
def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))
    val_ar.append(get_pixel(img, center, x-1, y + 1))
    val_ar.append(get_pixel(img, center, x, y + 1))
    val_ar.append(get_pixel(img, center, x + 1, y + 1))
    val_ar.append(get_pixel(img, center, x + 1, y))
    val_ar.append(get_pixel(img, center, x + 1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
  
    val = 0
     
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
         
    return val

In [ ]:
def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):  #iterate through each file 
        #print(image)
        
        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.
        
        img = dataset[image, :]
    ################################################################
    #START ADDING DATA TO THE DATAFRAME
  
                
         #Full image
        #GLCM = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        GLCM = greycomatrix(img, [1], [0])       
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df['Energy'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df['Corr'] = GLCM_corr       
        GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
        df['Diss_sim'] = GLCM_diss       
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df['Homogen'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df['Contrast'] = GLCM_contr


        GLCM2 = greycomatrix(img, [3], [0])       
        GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
        df['Energy2'] = GLCM_Energy2
        GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
        df['Corr2'] = GLCM_corr2       
        GLCM_diss2 = greycoprops(GLCM2, 'dissimilarity')[0]
        df['Diss_sim2'] = GLCM_diss2       
        GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
        df['Homogen2'] = GLCM_hom2       
        GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
        df['Contrast2'] = GLCM_contr2

        GLCM3 = greycomatrix(img, [5], [0])       
        GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
        df['Energy3'] = GLCM_Energy3
        GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
        df['Corr3'] = GLCM_corr3       
        GLCM_diss3 = greycoprops(GLCM3, 'dissimilarity')[0]
        df['Diss_sim3'] = GLCM_diss3       
        GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
        df['Homogen3'] = GLCM_hom3       
        GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
        df['Contrast3'] = GLCM_contr3

        GLCM4 = greycomatrix(img, [0], [np.pi/4])       
        GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
        df['Energy4'] = GLCM_Energy4
        GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
        df['Corr4'] = GLCM_corr4       
        GLCM_diss4 = greycoprops(GLCM4, 'dissimilarity')[0]
        df['Diss_sim4'] = GLCM_diss4       
        GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
        df['Homogen4'] = GLCM_hom4       
        GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
        df['Contrast4'] = GLCM_contr4
        
        GLCM5 = greycomatrix(img, [0], [np.pi/2])       
        GLCM_Energy5 = greycoprops(GLCM5, 'energy')[0]
        df['Energy5'] = GLCM_Energy5
        GLCM_corr5 = greycoprops(GLCM5, 'correlation')[0]
        df['Corr5'] = GLCM_corr5       
        GLCM_diss5 = greycoprops(GLCM5, 'dissimilarity')[0]
        df['Diss_sim5'] = GLCM_diss5       
        GLCM_hom5 = greycoprops(GLCM5, 'homogeneity')[0]
        df['Homogen5'] = GLCM_hom5       
        GLCM_contr5 = greycoprops(GLCM5, 'contrast')[0]
        df['Contrast5'] = GLCM_contr5
        
        #Add more filters as needed
        #entropy = shannon_entropy(img)
        #df['Entropy'] = entropy

        
        #Append features from current image to the dataset
        image_dataset = image_dataset.append(df)
        
    return image_dataset

### Lukas kanade

In [ ]:
dir='videos'
categories=['flood', 'agri', 'drought']
videos = []
labels = []
for category in categories:
    path=os.path.join(dir,category)
    label=categories.index(category)
    for img in os.listdir(path):
        imgpath=os.path.join(path,img)
        videos.append(imgpath)
        labels.append(label)

In [ ]:
labels[0]

0

In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

data = []
dynamic_features = []
frame_labels = []

for i in range(len(videos)):
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )

    cap = cv2.VideoCapture(videos[i])

    ret, old_frame = cap.read()

    # filters

    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # old_frame=cv2.resize(old_gray,(50,50))
    # gfilters = create_gaborfilter()
    # old_frame = apply_filter(old_frame, gfilters)

    old_frame=cv2.resize(old_gray,(50,50))
    # print(pet_img.shape)
    height, width = old_frame.shape
    # img_gray = cv2.cvtColor(pet_img,cv2.COLOR_BGR2GRAY)
    img_lbp_old = np.zeros((height, width),np.uint8)
    for k in range(0, height):
        for j in range(0, width):
            img_lbp_old[k, j] = lbp_calculated_pixel(old_frame, k, j)
    
    p0 = cv2.goodFeaturesToTrack(img_lbp_old, mask = None, **feature_params)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # filters

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # frame=cv2.resize(frame_gray,(50,50))
        # gfilters = create_gaborfilter()
        # frame = apply_filter(frame, gfilters)
        # data.append(frame)


        frame=cv2.resize(frame_gray,(50,50))
        # print(pet_img.shape)
        height, width = frame.shape
        # img_gray = cv2.cvtColor(pet_img,cv2.COLOR_BGR2GRAY)
        img_lbp_new = np.zeros((height, width),np.uint8)
        for k in range(0, height):
            for j in range(0, width):
                img_lbp_new[k, j] = lbp_calculated_pixel(frame, k, j)
        data.append(img_lbp_new)

        p1, st, err = cv2.calcOpticalFlowPyrLK(img_lbp_old, img_lbp_new, p0, None, **lk_params)

        flow_magnitude = np.sqrt(p1[:,:,0]**2 + p1[:,:,1]**2)
        dynamic_features.append(np.mean(flow_magnitude))
        # print(i)
        # print(labels[i])
        frame_labels.append(labels[i])
        # print(frame_labels)
        old_gray = frame_gray.copy()

    cap.release()

In [ ]:
data = np.array(data)

In [ ]:
data = feature_extractor(data)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
dynamic_features = pd.DataFrame(dynamic_features)

In [ ]:
df = pd.concat([dynamic_features.reset_index(drop=True), data.reset_index(drop=True)], axis=1)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(df, frame_labels, test_size=0.25,random_state=42)

In [ ]:
clf_svc = SVC()
clf_svc.fit(X_train, Y_train)
y_pred_svc = clf_svc.predict(X_test)
accuracy_svc = accuracy_score(Y_test, y_pred_svc)


clf_gaussian = GaussianNB()
clf_gaussian.fit(X_train, Y_train)
y_pred_gaussian = clf_gaussian.predict(X_test)
accuracy_gaussian = accuracy_score(Y_test, y_pred_gaussian)

clf_logistic = LogisticRegression()
clf_logistic.fit(X_train, Y_train)
y_pred_logistic = clf_logistic.predict(X_test)

print("===SVC===")
evaluate_model(Y_test, y_pred_svc)
print("===Gaussian===")
evaluate_model(Y_test, y_pred_gaussian)
print("===Logistic===")
evaluate_model(Y_test, y_pred_logistic)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

===SVC===
Accuracy: 0.63
Precision: 0.63
Recall: 0.63
F1 score: 0.63
===Gaussian===
Accuracy: 0.41
Precision: 0.41
Recall: 0.41
F1 score: 0.41
===Logistic===
Accuracy: 0.62
Precision: 0.62
Recall: 0.62
F1 score: 0.62


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


## gabor filter + local binary and lukas kanade

### gabor filter + local binary

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
  
     
def get_pixel(img, center, x, y):
     
    new_value = 0
     
    try:
        if img[x][y] >= center:
            new_value = 1
             
    except:
        pass
     
    return new_value
  
def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))
    val_ar.append(get_pixel(img, center, x-1, y + 1))
    val_ar.append(get_pixel(img, center, x, y + 1))
    val_ar.append(get_pixel(img, center, x + 1, y + 1))
    val_ar.append(get_pixel(img, center, x + 1, y))
    val_ar.append(get_pixel(img, center, x + 1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
  
    val = 0
     
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
         
    return val
  

In [ ]:
def create_gaborfilter():
    filters = []
    num_filters = 16
    ksize = 35 
    sigma = 3.0 
    lambd = 10.0
    gamma = 0.5
    psi = 0  
    for theta in np.arange(0, np.pi, np.pi / num_filters): 
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_64F)
        kern /= 1.0 * kern.sum()
        filters.append(kern)
    return filters

In [ ]:
def apply_filter(img, filters):
    newimage = np.zeros_like(img)
    depth = -1
    for kern in filters:
        image_filter = cv2.filter2D(img, depth, kern)
        np.maximum(newimage, image_filter, newimage)
    return newimage

In [ ]:
def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):  #iterate through each file 
        #print(image)
        
        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.
        
        img = dataset[image, :]
    ################################################################
    #START ADDING DATA TO THE DATAFRAME
  
                
         #Full image
        #GLCM = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        GLCM = greycomatrix(img, [1], [0])       
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df['Energy'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df['Corr'] = GLCM_corr       
        GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
        df['Diss_sim'] = GLCM_diss       
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df['Homogen'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df['Contrast'] = GLCM_contr


        GLCM2 = greycomatrix(img, [3], [0])       
        GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
        df['Energy2'] = GLCM_Energy2
        GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
        df['Corr2'] = GLCM_corr2       
        GLCM_diss2 = greycoprops(GLCM2, 'dissimilarity')[0]
        df['Diss_sim2'] = GLCM_diss2       
        GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
        df['Homogen2'] = GLCM_hom2       
        GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
        df['Contrast2'] = GLCM_contr2

        GLCM3 = greycomatrix(img, [5], [0])       
        GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
        df['Energy3'] = GLCM_Energy3
        GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
        df['Corr3'] = GLCM_corr3       
        GLCM_diss3 = greycoprops(GLCM3, 'dissimilarity')[0]
        df['Diss_sim3'] = GLCM_diss3       
        GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
        df['Homogen3'] = GLCM_hom3       
        GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
        df['Contrast3'] = GLCM_contr3

        GLCM4 = greycomatrix(img, [0], [np.pi/4])       
        GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
        df['Energy4'] = GLCM_Energy4
        GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
        df['Corr4'] = GLCM_corr4       
        GLCM_diss4 = greycoprops(GLCM4, 'dissimilarity')[0]
        df['Diss_sim4'] = GLCM_diss4       
        GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
        df['Homogen4'] = GLCM_hom4       
        GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
        df['Contrast4'] = GLCM_contr4
        
        GLCM5 = greycomatrix(img, [0], [np.pi/2])       
        GLCM_Energy5 = greycoprops(GLCM5, 'energy')[0]
        df['Energy5'] = GLCM_Energy5
        GLCM_corr5 = greycoprops(GLCM5, 'correlation')[0]
        df['Corr5'] = GLCM_corr5       
        GLCM_diss5 = greycoprops(GLCM5, 'dissimilarity')[0]
        df['Diss_sim5'] = GLCM_diss5       
        GLCM_hom5 = greycoprops(GLCM5, 'homogeneity')[0]
        df['Homogen5'] = GLCM_hom5       
        GLCM_contr5 = greycoprops(GLCM5, 'contrast')[0]
        df['Contrast5'] = GLCM_contr5
        
        #Add more filters as needed
        #entropy = shannon_entropy(img)
        #df['Entropy'] = entropy

        
        #Append features from current image to the dataset
        image_dataset = image_dataset.append(df)
        
    return image_dataset

### Lukas kanade

In [ ]:
dir='videos'
categories=['flood', 'agri', 'drought']
videos = []
labels = []
for category in categories:
    path=os.path.join(dir,category)
    label=categories.index(category)
    for img in os.listdir(path):
        imgpath=os.path.join(path,img)
        videos.append(imgpath)
        labels.append(label)

In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

data = []
dynamic_features = []
frame_labels = []

for i in range(len(videos)):
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )

    cap = cv2.VideoCapture(videos[i])

    ret, old_frame = cap.read()

    # filters

    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    old_frame=cv2.resize(old_gray,(50,50))
    gfilters = create_gaborfilter()
    old_frame = apply_filter(old_frame, gfilters)

    height, width = old_frame.shape
    # img_gray = cv2.cvtColor(pet_img,cv2.COLOR_BGR2GRAY)
    img_lbp_old = np.zeros((height, width),np.uint8)
    for k in range(0, height):
        for j in range(0, width):
            img_lbp_old[k, j] = lbp_calculated_pixel(old_frame, k, j)

    
    p0 = cv2.goodFeaturesToTrack(img_lbp_old, mask = None, **feature_params)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # filters

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame=cv2.resize(frame_gray,(50,50))
        gfilters = create_gaborfilter()
        frame = apply_filter(frame, gfilters)

        height, width = frame.shape
        # img_gray = cv2.cvtColor(pet_img,cv2.COLOR_BGR2GRAY)
        img_lbp_new = np.zeros((height, width),np.uint8)
        for k in range(0, height):
            for j in range(0, width):
                img_lbp_new[k, j] = lbp_calculated_pixel(frame, k, j)
        data.append(img_lbp_new)

        p1, st, err = cv2.calcOpticalFlowPyrLK(img_lbp_old, img_lbp_new, p0, None, **lk_params)

        flow_magnitude = np.sqrt(p1[:,:,0]**2 + p1[:,:,1]**2)
        dynamic_features.append(np.mean(flow_magnitude))
        frame_labels.append(labels[i])
        old_gray = frame_gray.copy()

    cap.release()

In [ ]:
data = np.array(data)

In [ ]:
data = feature_extractor(data)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
dynamic_features = pd.DataFrame(dynamic_features)

In [ ]:
df = pd.concat([dynamic_features.reset_index(drop=True), data.reset_index(drop=True)], axis=1)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(df, frame_labels, test_size=0.25,random_state=42)

In [ ]:
clf_svc = SVC()
clf_svc.fit(X_train, Y_train)
y_pred_svc = clf_svc.predict(X_test)
accuracy_svc = accuracy_score(Y_test, y_pred_svc)


clf_gaussian = GaussianNB()
clf_gaussian.fit(X_train, Y_train)
y_pred_gaussian = clf_gaussian.predict(X_test)
accuracy_gaussian = accuracy_score(Y_test, y_pred_gaussian)

clf_logistic = LogisticRegression()
clf_logistic.fit(X_train, Y_train)
y_pred_logistic = clf_logistic.predict(X_test)

print("===SVC===")
evaluate_model(Y_test, y_pred_svc)
print("===Gaussian===")
evaluate_model(Y_test, y_pred_gaussian)
print("===Logistic===")
evaluate_model(Y_test, y_pred_logistic)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

===SVC===
Accuracy: 0.61
Precision: 0.61
Recall: 0.61
F1 score: 0.61
===Gaussian===
Accuracy: 0.52
Precision: 0.52
Recall: 0.52
F1 score: 0.52
===Logistic===
Accuracy: 0.56
Precision: 0.56
Recall: 0.56
F1 score: 0.56


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
